# Rank Most Actors By Genre

Using various distance metrics, identify the most similar actors to a given query actor by the genres in which they've starred.

In [2]:
%matplotlib inline


In [3]:
import pandas as pd

import json

import matplotlib.pyplot as plt

In [4]:
actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}


with open("../data/imdb_movies_2000to2022.prolific.json", "r") as in_file:
    for line in in_file:
        
        # Read the movie on this line and parse its json
        this_movie = json.loads(line)
                    
        # Add all actors to the id->name map
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        # For each actor, add this movie's genres to that actor's list
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            # Increment the count of genres for this actor
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            # Update the map
            actor_genre_map[actor_id] = this_actors_genres
            
        # Finished with this film
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"]
        })

In [5]:
# Check the output for actor Hugh Jackman, actor ID nm0413168
actor_genre_map['nm0413168']

{'Comedy': 7,
 'Fantasy': 3,
 'Romance': 5,
 'Action': 14,
 'Adventure': 11,
 'Sci-Fi': 10,
 'Crime': 6,
 'Thriller': 2,
 'Animation': 4,
 'Drama': 12,
 'Mystery': 5,
 'Biography': 4,
 'Musical': 2,
 'History': 1}

In [6]:
# Get all actors as an index for a dataframe
index = actor_genre_map.keys()

# Get the genre-counts for each actor in the index
rows = [actor_genre_map[k] for k in index]

# Create the data frame from these rows, with the actors as index
df = pd.DataFrame(rows, index=index)

# Fill NAs with zero, as NA means the actor has not starred in that genre
df = df.fillna(0)

df

,Comedy,Fantasy,Romance,Drama,Mystery,Thriller,Action,Biography,Crime,War,...,Horror,Documentary,Sport,News,Family,Music,,Western,Short,Reality-TV
nm0000212,7.0,1.0,6.0,6.0,1.0,2.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0413168,7.0,3.0,5.0,12.0,5.0,2.0,14.0,4.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0000630,8.0,2.0,6.0,14.0,2.0,3.0,4.0,5.0,1.0,1.0,...,3.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0005227,10.0,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
nm0864851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nm9504284,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm10592896,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm7216750,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0936300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Normalize Rows

We normalized rows by dividing through the sum of each row. This transformation, L1-normalization, changes rows from counts to proportions.

In [7]:
df_norm = df.divide(df.sum(axis=1), axis=0)
df_norm.head(10)

,Comedy,Fantasy,Romance,Drama,Mystery,Thriller,Action,Biography,Crime,War,...,Horror,Documentary,Sport,News,Family,Music,,Western,Short,Reality-TV
nm0000212,0.250000,0.035714,0.214286,0.214286,0.035714,0.071429,0.035714,0.035714,0.071429,0.035714,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0413168,0.081395,0.034884,0.058140,0.139535,0.058140,0.023256,0.162791,0.046512,0.069767,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0000630,0.112676,0.028169,0.084507,0.197183,0.028169,0.042254,0.056338,0.070423,0.014085,0.014085,...,0.042254,0.098592,0.042254,0.014085,0.00,0.0,0.0,0.0,0.0,0.0
nm0005227,0.400000,0.040000,0.080000,0.080000,0.000000,0.040000,0.040000,0.000000,0.000000,0.000000,...,0.040000,0.000000,0.040000,0.000000,0.08,0.0,0.0,0.0,0.0,0.0
nm0864851,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0828288,0.119403,0.000000,0.000000,0.059701,0.044776,0.149254,0.014925,0.000000,0.044776,0.000000,...,0.522388,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0933983,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0329491,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0000417,0.119048,0.023810,0.000000,0.190476,0.071429,0.095238,0.119048,0.000000,0.095238,0.000000,...,0.119048,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
nm0000603,0.062500,0.031250,0.062500,0.375000,0.062500,0.062500,0.000000,0.093750,0.062500,0.000000,...,0.031250,0.031250,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0


In [8]:
df_norm.sum(axis=1).head(20)

nm0000212    1.0
nm0413168    1.0
nm0000630    1.0
nm0005227    1.0
nm0864851    1.0
nm0828288    1.0
nm0933983    1.0
nm0329491    1.0
nm0000417    1.0
nm0000603    1.0
nm0000457    1.0
nm0452288    1.0
nm0001002    1.0
nm0001299    1.0
nm0923529    1.0
nm0936365    1.0
nm0006763    1.0
nm0007113    1.0
nm0310173    1.0
nm0412917    1.0
dtype: float64

## Actor Similarity Using Counts

In [9]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [10]:
#Setting the actor we will be comparing to
# target_actor_id = 'nm1165110' # Chris Hemsworth
target_actor_id = 'nm0413168' # Hugh Jackman
# target_actor_id = 'nm0005351' # Ryan Reynolds

In [20]:
#Gathering the genres for that actor
target_actor_ratings = df.loc[target_actor_id]

#Generating distances from that actor to all the others
distances = scipy.spatial.distance.cdist(df, [target_actor_ratings], metric="cosine")[:,0]

query_distances = list(zip(df.index, distances))

#Printing the top ten most similar actors to our target
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    print(similar_actor_id, actor_name_map[similar_actor_id], similar_genre_score, df.loc[similar_actor_id].sum())

nm0413168 Hugh Jackman 0.0 86.0
nm3592338 Emilia Clarke 0.023038353871383088 20.0
nm1663205 Sharlto Copley 0.03680695049665894 39.0
nm0000375 Robert Downey Jr. 0.04015740464249473 75.0
nm3772243 Theo James 0.04358329306959263 46.0
nm0262635 Chris Evans 0.04805589651668307 74.0
nm0881631 Karl Urban 0.05508543978437186 44.0
nm0159789 Hayden Christensen 0.060195990212304484 46.0
nm1517976 Chris Pine 0.07953201413353528 66.0
nm1475594 Channing Tatum 0.08366839174462504 72.0


In [19]:
#Gathering the genres for that actor
target_actor_ratings = df.loc[target_actor_id]

#Generating distances from that actor to all the others
distances = scipy.spatial.distance.cdist(df, [target_actor_ratings], metric="euclidean")[:,0]

query_distances = list(zip(df.index, distances))

#Printing the top ten most similar actors to our target
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    print(similar_actor_id, actor_name_map[similar_actor_id], similar_genre_score, df.loc[similar_actor_id].sum())

nm0413168 Hugh Jackman 0.0 86.0
nm0000375 Robert Downey Jr. 7.937253933193772 75.0
nm0262635 Chris Evans 8.366600265340756 74.0
nm1517976 Chris Pine 10.770329614269007 66.0
nm1475594 Channing Tatum 10.954451150103322 72.0
nm0185819 Daniel Craig 11.445523142259598 73.0
nm0000226 Will Smith 11.916375287812984 64.0
nm0757855 Zoe Saldana 12.328828005937952 66.0
nm0004937 Jamie Foxx 12.529964086141668 67.0
nm0000234 Charlize Theron 12.922847983320086 95.0


## Calculate Euclidean Distance in L1-Normed Space

Above, we see differences in cosine and Euclidean distances. Below, we apply Euclidean distance to the normalized matrix, and we see results much more similar to cosine distance.

In [21]:
#Gathering the genres for that actor
target_actor_ratings = df_norm.loc[target_actor_id]

#Generating distances from that actor to all the others
distances = scipy.spatial.distance.cdist(df_norm, [target_actor_ratings], metric="euclidean")[:,0]

query_distances = list(zip(df.index, distances))

#Printing the top ten most similar actors to our target
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    print(similar_actor_id, actor_name_map[similar_actor_id], similar_genre_score, df.loc[similar_actor_id].sum())

nm0413168 Hugh Jackman 0.0 86.0
nm3592338 Emilia Clarke 0.07107305483691222 20.0
nm1663205 Sharlto Copley 0.08967075576182816 39.0
nm0000375 Robert Downey Jr. 0.09072324558361769 75.0
nm3772243 Theo James 0.09798239863947802 46.0
nm0881631 Karl Urban 0.10799068657869043 44.0
nm0262635 Chris Evans 0.10987243021473778 74.0
nm0159789 Hayden Christensen 0.11462756498093742 46.0
nm0000168 Samuel L. Jackson 0.1309070605743363 155.0
nm0005212 Ian McKellen 0.1329223499989793 51.0
